# 初始化目录

In [54]:
# 加入plot支持
%matplotlib inline

# 把当前路径的上级路径作为root
import os
import sys
cur_dir = os.getcwd()
pkg_rootdir = os.path.dirname(cur_dir)
sys.path.append(pkg_rootdir)
os.chdir(pkg_rootdir)
print("当前工作路径：",pkg_rootdir)

当前工作路径： /


In [55]:
from mlstock.data import data_filter
df = data_filter.filter_stocks()
print(df)

        ts_code     name  area industry market list_status list_date  \
0     000001.SZ     平安银行    深圳       银行     主板           L  19910403   
1     000002.SZ      万科A    深圳     全国地产     主板           L  19910129   
2     000004.SZ     ST国华    深圳     软件服务     主板           L  19910114   
3     000005.SZ     ST星源    深圳     环境保护     主板           L  19901210   
4     000006.SZ     深振业A    深圳     区域地产     主板           L  19920427   
...         ...      ...   ...      ...    ...         ...       ...   
4826  871981.BJ     晶赛科技  None     None    北交所           L  20211115   
4827  872925.BJ     锦好医疗  None     None    北交所           L  20211025   
4828  873169.BJ     七丰精工  None     None    北交所           L  20220415   
4829  873223.BJ     荣亿精密  None     None    北交所           L  20220609   
4830  689009.SH  九号公司-WD    北京      摩托车    CDR           L  20201029   

     delist_date  
0           None  
1           None  
2           None  
3           None  
4           None  
...          ...  
48

# 过滤股票集合

In [60]:
from mlstock.ml.main import main
df = main("20180101","20220101",20)

        ts_code     name  area industry market list_status list_date  \
0     000001.SZ     平安银行    深圳       银行     主板           L  19910403   
1     000002.SZ      万科A    深圳     全国地产     主板           L  19910129   
2     000004.SZ     ST国华    深圳     软件服务     主板           L  19910114   
3     000005.SZ     ST星源    深圳     环境保护     主板           L  19901210   
4     000006.SZ     深振业A    深圳     区域地产     主板           L  19920427   
...         ...      ...   ...      ...    ...         ...       ...   
4826  871981.BJ     晶赛科技  None     None    北交所           L  20211115   
4827  872925.BJ     锦好医疗  None     None    北交所           L  20211025   
4828  873169.BJ     七丰精工  None     None    北交所           L  20220415   
4829  873223.BJ     荣亿精密  None     None    北交所           L  20220609   
4830  689009.SH  九号公司-WD    北京      摩托车    CDR           L  20201029   

     delist_date  
0           None  
1           None  
2           None  
3           None  
4           None  
...          ...  
48

In [58]:
df

# 划分训练测试集

In [42]:
trade_dates = df.trade_date.sort_values().unique()

AttributeError: 'NoneType' object has no attribute 'trade_date'

In [43]:
import math
div_num = math.ceil(len(trade_dates)*0.8)
train_dates = trade_dates[:div_num]
test_dates =  trade_dates[div_num:]
df_train = df[df.trade_date.apply(lambda x: x in train_dates)]
df_test = df[df.trade_date.apply(lambda x: x in test_dates)]

df_test.pct_chg_hs300.max()
# df_test.shape

NameError: name 'trade_dates' is not defined

In [44]:
df_train

NameError: name 'df_train' is not defined

# 去除上市12周内的数据（太剧烈波动）

In [45]:
import pandas as pd
a = pd.to_datetime(df_train.trade_date, format='%Y%m%d')
b = pd.to_datetime(df_train.list_date, format='%Y%m%d')
df_train = df_train[a-b > pd.Timedelta(12,unit='w')] 
df_train

NameError: name 'df_train' is not defined

In [46]:
a = pd.to_datetime(df_test.trade_date, format='%Y%m%d')
b = pd.to_datetime(df_test.list_date, format='%Y%m%d')
df_test = df_test[a-b > pd.Timedelta(12,unit='w')] 
df_test

NameError: name 'df_test' is not defined

In [47]:
print(df_train.target.count(),df_train.shape[0],df_train.target.count()/df_train.shape[0])
df_train = df_train[~df_train.target.isnull()]
df_test = df_test[~df_test.target.isnull()]
print(df_train.target.count(),df_train.shape[0],df_train.target.count()/df_train.shape[0])

NameError: name 'df_train' is not defined

# 去极值

In [48]:
# 保留feature
feature_names = ['MACD','KDJ']
df_features = df_train[feature_names]
# 每列都求中位数，和中位数之差的绝对值的中位数
df_median = df_features.median()
df_scope = df_features.apply(lambda x: x - df_median[x.name]).abs().median()
df_scope
def scaller(x):
    _max = df_median[x.name] + 5 * df_scope[x.name]
    _min = df_median[x.name] - 5 * df_scope[x.name]
    x = x.apply(lambda v: _min if v < _min else v)
    x = x.apply(lambda v: _max if v > _max else v)
    return x
df_features = df_features.apply(scaller)
df_train[feature_names] = df_features
df_train

NameError: name 'df_train' is not defined

# 空值按照行业平均填充 TODO

In [49]:
# df_features_mean = df_train.groupby(by='industry').mean()
# df_features_mean
# df_mean_industry = df_mean_industry.fillna(df_mean_industry.mean())
# df_mean_industry.columns = [x+'_mean' for x in df_mean_industry.columns]
# df_mean_industry = df_mean_industry.reset_index()

# 标准化

将中性化处理后的因子暴露度序列减去其现在的均值、除以其标准差，得到一个新的近似服从N(0,1)分布的序列。
所以要对一个列中的所有的数据做，本来考虑用行业，但是后来思考后，觉得整体服从正态分布更重要。

In [50]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df_train[feature_names])
df_train[feature_names] = scaler.transform(df_train[feature_names])
df_test[feature_names] = scaler.transform(df_test[feature_names])
df_train

NameError: name 'df_train' is not defined

# 训练

In [51]:
df_train

NameError: name 'df_train' is not defined

In [52]:
from sklearn import linear_model

df_train.dropna(subset=feature_names,inplace=True)
df_test.dropna(subset=feature_names,inplace=True)

X_train = df_train[feature_names+['target']].values
X_test = df_test[feature_names+['target']].values
y_train = df_train.target
y_test = df_test.target

reg = linear_model.LinearRegression()
print(X_train.shape,y_train.shape)
model = reg.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred

NameError: name 'df_train' is not defined

# 模型评价

In [36]:
df_result = pd.DataFrame({'ts_code':df_test.ts_code, 
                          'trade_date':df_test.trade_date,
                          'y':y_test, 
                          'y_pred':y_pred})
ic = df_result[['y', 'y_pred']].corr().iloc[0,1]

df_result['y_rank'] = df_result.y.rank(ascending=False) # 并列的默认使用排名均值
df_result['y_pred_rank'] = df_result.y_pred.rank(ascending=False) 

rank_ic = df_result[['y_rank', 'y_pred_rank']].corr().iloc[0,1]

print("ic,rank_ic:",(ic,rank_ic))

NameError: name 'pd' is not defined

# 策略评价

## 分层回测

In [30]:
df_result['industry'] = df_test.industry 
df_result['y_rank_idst'] = df_result.groupby('industry').y_pred.rank(ascending=False)

NameError: name 'df_test' is not defined

In [23]:
df_result

NameError: name 'df_result' is not defined